<a href="https://colab.research.google.com/github/Mahdi-Saadati/Advanced-Machine-Learning-and-Deep-Neural-Networks/blob/main/9.a.%20Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" align="right">

شبکه ترنسفورمر (Transformer)

1️⃣ ایده اصلی

فرض کنید می‌خواهیم یک متن را پردازش کنیم، مثلاً جمله:
"من امروز به مدرسه رفتم"

هدف ما این است که مدل بتواند بفهمد ارتباط بین کلمات چیست، حتی اگر کلمات فاصله زیادی داشته باشند.

قبل از Transformer، مدل‌های قدیمی مثل RNN یا LSTM کلمات را یکی یکی پردازش می‌کردند و اگر یک کلمه خیلی دور بود، یادگیری ارتباط سخت می‌شد.

Transformer می‌گوید:

"چرا همه چیز را یکجا نبینیم و مستقیماً ارتباط بین هر کلمه با هر کلمه را بسنجیم؟"

2️⃣ بخش‌های کلیدی Transformer
🔹 ورودی: کلمات → بردارها

هر کلمه تبدیل می‌شود به یک عدد یا بردار عددی (مثل شناسه کلمه یا embedding).
مثال: "من" → [0.1, 0.3, 0.5]
"مدرسه" → [0.7, 0.2, 0.9]

🔹 Attention (توجه)

اینجاست که جادو اتفاق می‌افتد:

به هر کلمه می‌گوید: "به کدام کلمات دیگر جمله توجه کنم تا معنی‌ام را بهتر بفهمم؟"

مثال:

کلمه "رفتم" بیشتر به "مدرسه" توجه می‌کند تا "امروز"، چون معنی رفتن به مقصد مرتبط است.

این کار با محاسبه وزن‌ها بین همه کلمات انجام می‌شود.

وزن زیاد = ارتباط قوی

وزن کم = ارتباط ضعیف

این بخش Self-Attention نام دارد.

🔹 Feed-Forward (لایه پردازشی)

بعد از Attention، هر کلمه با اطلاعاتی که از دیگر کلمات گرفته ترکیب می‌شود و از چند لایه ساده پردازش عبور می‌کند تا اطلاعات بهتر یاد گرفته شود.

🔹 Positional Encoding

چون Transformer همه کلمات را همزمان پردازش می‌کند، باید بداند ترتیب کلمات چیست.

این کار با اضافه کردن یک بردار موقعیت به هر کلمه انجام می‌شود.
مثال:
"من" → [0.1,0.3,0.5] + position_0
"مدرسه" → [0.7,0.2,0.9] + position_3

🔹 Encoder و Decoder

Encoder: متن ورودی را می‌فهمد و اطلاعات مهم را استخراج می‌کند.

Decoder: بر اساس اطلاعات Encoder، متن خروجی می‌سازد (مثل ترجمه یا پاسخ).

3️⃣ خلاصه بصری ساده
ورودی متن → تبدیل به بردار → Attention (هر کلمه به همه نگاه می‌کند)
→ لایه‌های پردازش → خروجی معنایی (یا ترجمه، یا پیش‌بینی کلمه بعدی)


بزرگترین قدرت Transformer این است که می‌تواند ارتباطات دور بین کلمات را به راحتی یاد بگیرد، بدون نیاز به پردازش کلمه به کلمه.

💡 مثال روزمره:
فرض کن می‌خواهی بفهمی جمله‌ی "علی که دیروز مریض بود امروز به مدرسه رفت" چه معنایی دارد.

Transformer می‌تواند بفهمد "رفتن" مربوط به "علی" و "مدرسه" است، حتی اگر کلمه‌ها فاصله زیادی داشته باشند.

In [1]:
from keras.layers import TextVectorization, Input, Embedding, MultiHeadAttention, LayerNormalization, Dense
from keras.models import Model
import numpy as np

# 1. داده‌های اولیه: جملات انگلیسی و ترجمه فارسی
eng_texts = [
    "I love AI",                       # جمله انگلیسی
    "Deep learning is fun",            # جمله انگلیسی
    "Transformers are powerful",       # جمله انگلیسی
]
fa_texts = [
    "من هوش مصنوعی را دوست دارم",        # ترجمه فارسی
    "یادگیری عمیق جذاب است",             # ترجمه فارسی
    "ترنسفورمرها قدرتمند هستند",          # ترجمه فارسی
]

# 2. آماده‌سازی ابزار بردارسازی متن (تبدیل کلمات → اعداد)
max_len = 6  # حداکثر طول جمله (کلمات اضافه کوتاه یا پر می‌شوند)
eng_vectorizer = TextVectorization(max_tokens=50, output_sequence_length=max_len)
eng_vectorizer.adapt(eng_texts)  # ساخت واژگان انگلیسی
fa_vectorizer = TextVectorization(max_tokens=50, output_sequence_length=max_len)
fa_vectorizer.adapt(fa_texts)    # ساخت واژگان فارسی

# واژگان (دیکشنری لغات)
eng_vocab = eng_vectorizer.get_vocabulary()
fa_vocab = fa_vectorizer.get_vocabulary()

# 3. تبدیل جملات به توالی عددی
eng_seq = eng_vectorizer(eng_texts).numpy()[:, :-1]  # ورودی مدل (encoder)
fa_seq = fa_vectorizer(fa_texts).numpy()[:, 1:]      # خروجی مدل (decoder target, شیفت شده)

# 4. تعریف مدل Transformer ساده
embed_dim = 16   # اندازه بردار هر کلمه
num_heads = 2    # تعداد هد در Multi-Head Attention
ff_dim = 32      # اندازه لایه Feed Forward

# --- Encoder ---
encoder_inputs = Input(shape=(max_len-1,))  # ورودی جملات انگلیسی (اعداد)
x = Embedding(input_dim=len(eng_vocab), output_dim=embed_dim)(encoder_inputs)  # تبدیل عدد → بردار
attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x) # Self-Attention روی انگلیسی
x = LayerNormalization(epsilon=1e-6)(x + attn_output)  # نرمال‌سازی + اتصال باقی‌مانده (residual)
ff = Dense(ff_dim, activation="relu")(x)   # لایه Feed Forward
ff = Dense(embed_dim)(ff)
encoder_outputs = LayerNormalization(epsilon=1e-6)(x + ff)  # خروجی نهایی encoder

# --- Decoder ---
decoder_inputs = Input(shape=(max_len-1,))  # ورودی فارسی (shift شده)
y = Embedding(input_dim=len(fa_vocab), output_dim=embed_dim)(decoder_inputs)  # عدد → بردار
attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(y, encoder_outputs) # توجه به encoder
y = LayerNormalization(epsilon=1e-6)(y + attn_output)
ff = Dense(ff_dim, activation="relu")(y)
ff = Dense(embed_dim)(ff)
decoder_outputs = LayerNormalization(epsilon=1e-6)(y + ff)  # خروجی decoder

# --- Output layer ---
outputs = Dense(len(fa_vocab), activation="softmax")(decoder_outputs)  # پیش‌بینی کلمات فارسی

# ساخت و کامپایل مدل
model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()  # نمایش ساختار مدل

# 5. آموزش مدل روی داده‌های کوچک (500 تکرار برای نمونه کوچک)
model.fit([eng_seq, fa_seq], np.expand_dims(fa_seq, -1), epochs=500, verbose=0)

# 6. تست مدل روی یک جمله جدید
test_sentence = "I love AI"  # جمله تست
test_seq = eng_vectorizer([test_sentence]).numpy()[:, :-1]  # تبدیل به عدد
decoder_input = np.zeros((1, max_len-1))  # ورودی اولیه decoder (شروع با صفرها)
pred = model.predict([test_seq, decoder_input])  # پیش‌بینی کلمه به کلمه

# انتخاب بیشترین احتمال برای هر کلمه
pred_indices = np.argmax(pred[0], axis=-1)
# بازگرداندن اعداد → کلمات فارسی
pred_words = [fa_vocab[i] for i in pred_indices if i != 0]
translation = " ".join(pred_words)

print("English:", test_sentence)
print("Predicted Persian:", translation)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 5, 16)     │        192 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 16)     │      2,160 │ embedding[0][0],  │
│ (MultiHeadAttentio… │                   │            │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 5, 16)     │          0 │ embedding[0][0],  │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 5, 16)     │         32 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5, 32)     │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5, 16)     │        528 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 5, 16)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 5, 16)     │        240 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 16)     │      2,160 │ embedding_1[0][0… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 5, 16)     │          0 │ embedding_1[0][0… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 5, 32)     │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5, 16)     │        528 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 5, 16)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 7,279 (28.43 KB)

 Trainable params: 7,279 (28.43 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
English: I love AI
Predicted Persian: 
